# Visualization reducing grammar

## Inspecting the corpora

1. Sentence length
2. Vocabsize
3. Amount of terminals
4. Amount of non terminals
5. Distribution (Zipf)
6. Train/test/dev

#### Per top k

In [2]:
# open txt file
version = 'normal'
top_k = '0.2'
with open(f'corpora/{version}/train_sent_{top_k}.txt', 'r') as file:
    data = file.read().replace('\n', '')